# 01-femsainfancias_missingcities_osmnxnetwork

This notebook runs the missing cities of project femsainfancias, creates a Network (G,nodes,edges) for a bbox around the city and saves it to db.

## Import libraries

In [1]:
import os
import sys

import pandas as pd
import geopandas as gpd
import osmnx as ox
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

module_path = os.path.abspath(os.path.join('../../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

In [3]:
##### WARNING ##### WARNING ##### WARNING ##### WARNING ##### WARNING #####

# save output to database?
save = True

##### WARNING ##### WARNING ##### WARNING ##### WARNING ##### WARNING #####

## Load cities

In [2]:
# gdf_mun
metro_schema = 'projects_research'
metro_table = 'femsainfancias_missingcities_metrogdf2020'

query = f"SELECT * FROM {metro_schema}.{metro_table}"
metro_gdf = aup.gdf_from_query(query, geometry_col='geometry')
metro_gdf = metro_gdf.set_crs("EPSG:4326")

city_list = list(metro_gdf.city.unique())

# Show
print(metro_gdf.shape)
print(len(city_list))
print(city_list)
metro_gdf.head(1)

(4, 6)
2
['Zihuatanejo', 'San Cristobal']


,CVEGEO,CVE_ENT,CVE_MUN,NOMGEO,city,geometry
0,12038,12,038,Zihuatanejo de Azueta,Zihuatanejo,"MULTIPOLYGON (((-101.65842 17.67894, -101.6580..."


## Run function create_osmnx_network for each city and save

In [4]:
save_schema = 'projects_research'
nodes_save_table = 'femsainfancias_missingcities_nodes'
edges_save_table = 'femsainfancias_missingcities_edges'

In [9]:
k = len(city_list)

# prevent cities being analyzed several times in case of a crash
nodes_processed_city_list = []
edges_processed_city_list = []

try:
    query = f"SELECT city FROM {save_schema}.{nodes_save_table}"
    nodes_processed = aup.df_from_query(query)
    nodes_processed_city_list = list(nodes_processed.city.unique())
    
    query = f"SELECT city FROM {save_schema}.{edges_save_table}"
    edges_processed = aup.df_from_query(query)
    edges_processed_city_list = list(edges_processed.city.unique())

except:
    pass

# PRINT PROGRESS
missing_cities_nodes = []
for city in city_list:
    if city not in nodes_processed_city_list:
        missing_cities_nodes.append(city)

i = len(nodes_processed_city_list)
print(f'Already processed nodes for ({i}/{k}) cities.')
print(f'Missing nodes procesing for cities: {missing_cities_nodes}')

missing_cities_edges = []
for city in city_list:
    if city not in edges_processed_city_list:
        missing_cities_edges.append(city)

j = len(nodes_processed_city_list)
print(f'Already processed nodes for ({i}/{k}) cities.')
print(f'Missing nodes procesing for cities: {missing_cities_nodes}')    

# SKIP FAILED CITIES
skip_city_list = []

# RUN ALL OTHER CITIES
for city in city_list:
    if city not in skip_city_list:
        if (city not in nodes_processed_city_list) and (city not in edges_processed_city_list):
            print("--"*40)
            i = i + 1
            print(f"Starting city {i}/{k}: {city}")
        
            # Load area of interest
            city_gdf = metro_gdf.loc[metro_gdf.city == city]
            aoi = city_gdf.dissolve()
            
            # Create Network
            G,nodes,edges = aup.create_osmnx_network(aoi, how='from_bbox', network_type='all_private')

            nodes['city'] = city
            edges['city'] = city
        
            # Save Network
            if save:

                # Resets index (allows delete duplicates in DBeaver using u, v, osmid)
                nodes = nodes.reset_index()
                edges = edges.reset_index()
                
                print(f"Uploading nodes for {city}")
                aup.gdf_to_db_slow(nodes, nodes_save_table, save_schema, if_exists='append')
                print(f"Uploading edges for {city}")
                aup.gdf_to_db_slow(edges, edges_save_table, save_schema, if_exists='append')

        else:
            print("--"*40)
            print(f"{city} already processed. ({i}/{k})")
            
    else:
        print("--"*40)
        print(f"SKIPPED {city}")

Already processed nodes for (0/2) cities.
Missing nodes procesing for cities: ['Zihuatanejo', 'San Cristobal']
Already processed nodes for (0/2) cities.
Missing nodes procesing for cities: ['Zihuatanejo', 'San Cristobal']
--------------------------------------------------------------------------------
Starting city 1/2: Zihuatanejo
Extracted min and max coordinates from the municipality. Polygon N:18.06141, S:17.55477, E-101.20053, W-101.71494.
Created OSMnx graph from bounding box.
Converted OSMnx graph to 10802 nodes and 26727 edges GeoDataFrame.
Added column width for edges.
Filtered columns.
Column: osmid in nodes gdf, has a list in it, the column data was converted to string.
Column: name in nodes gdf, has a list in it, the column data was converted to string.
Column: highway in nodes gdf, has a list in it, the column data was converted to string.
Uploading nodes for Zihuatanejo
Uploading edges for Zihuatanejo
-----------------------------------------------------------------------

## Tests

### Load all metro, nodes, edges

In [12]:
# gdf_mun
metro_schema = 'projects_research'
metro_table = 'femsainfancias_missingcities_metrogdf2020'

query = f"SELECT * FROM {metro_schema}.{metro_table}"
metro_gdf = aup.gdf_from_query(query, geometry_col='geometry')
metro_gdf = metro_gdf.set_crs("EPSG:4326")

# Show
print(metro_gdf.shape)
metro_gdf.head(1)

(4, 6)


,CVEGEO,CVE_ENT,CVE_MUN,NOMGEO,city,geometry
0,12038,12,038,Zihuatanejo de Azueta,Zihuatanejo,"MULTIPOLYGON (((-101.65842 17.67894, -101.6580..."


In [9]:
# nodes
nodes_schema = 'projects_research'
nodes_table = 'femsainfancias_missingcities_nodes'

query = f"SELECT * FROM {nodes_schema}.{nodes_table}"
nodes_gdf = aup.gdf_from_query(query, geometry_col='geometry')
nodes_gdf = nodes_gdf.set_crs("EPSG:4326")

# Show
print(nodes_gdf.shape)
nodes_gdf.head(1)

(28618, 6)


,osmid,x,y,street_count,geometry,city
0,308237923,-101.345647,17.567709,4,POINT (-101.34565 17.56771),Zihuatanejo


In [8]:
# edges
edges_schema = 'projects_research'
edges_table = 'femsainfancias_missingcities_edges'

query = f"SELECT * FROM {edges_schema}.{edges_table}"
edges_gdf = aup.gdf_from_query(query, geometry_col='geometry')
edges_gdf = edges_gdf.set_crs("EPSG:4326")

# Show
print(edges_gdf.shape)
edges_gdf.head(1)

(71538, 19)


,u,v,key,osmid,oneway,lanes,name,highway,maxspeed,length,geometry,bridge,ref,junction,tunnel,access,width,service,city
0,308237923,4320069487,0,433122830,False,None,None,residential,None,13.918,"LINESTRING (-101.34565 17.56771, -101.34572 17...",None,None,None,None,None,NaN,None,Zihuatanejo


### City test

In [28]:
city_test = 'Zihuatanejo'

In [29]:
metro_test = metro_gdf.loc[metro_gdf.city==city_test]
metro_test.head(1)

,CVEGEO,CVE_ENT,CVE_MUN,NOMGEO,city,geometry
0,12038,12,038,Zihuatanejo de Azueta,Zihuatanejo,"MULTIPOLYGON (((-101.65842 17.67894, -101.6580..."


In [30]:
nodes_test = nodes_gdf.loc[nodes_gdf.city==city_test]
nodes_test.head(1)

,osmid,x,y,street_count,geometry,city
0,308237923,-101.345647,17.567709,4,POINT (-101.34565 17.56771),Zihuatanejo


In [31]:
edges_test = edges_gdf.loc[edges_gdf.city==city_test]
edges_test.head(1)

,u,v,key,osmid,oneway,lanes,name,highway,maxspeed,length,geometry,bridge,ref,junction,tunnel,access,width,service,city
0,308237923,4320069487,0,433122830,False,None,None,residential,None,13.918,"LINESTRING (-101.34565 17.56771, -101.34572 17...",None,None,None,None,None,NaN,None,Zihuatanejo


In [32]:
G, nodes, edges = aup.graph_from_hippo(metro_test, schema='projects_research', edges_folder='femsainfancias_missingcities_edges', nodes_folder='femsainfancias_missingcities_nodes')

In [37]:
# Show
edges.head(5)

osmid  oneway lanes   
u         v          key                                         
308237994 8082848179 0                 867447567   False     2  \
          8085344555 0                 867141997   False  None   
          8082848201 0    [190128099, 867447567]   False     2   
308237995 8097498158 0                 867141999   False     2   
          8085782154 0                 867141999   False     2   

                                                              name   
u         v          key                                             
308237994 8082848179 0    Carretera Federal Acapulco - Zihuatanejo  \
          8085344555 0                                        None   
          8082848201 0    Carretera Federal Acapulco - Zihuatanejo   
308237995 8097498158 0    Carretera Federal Acapulco - Zihuatanejo   
          8085782154 0    Carretera Federal Acapulco - Zihuatanejo   

                              highway maxspeed   length   
u         v          key                                  
308237994 8082848179 0          trunk     None  141.252  \
          8085344555 0    residential     None   15.925   
          8082848201 0          trunk     None  737.709   
308237995 8097498158 0          trunk     None    4.044   
          8085782154 0          trunk     None   78.769   

                                                                   geometry   
u         v          key                                                      
308237994 8082848179 0    LINESTRING (-101.41440 17.60000, -101.41567 17...  \
          8085344555 0    LINESTRING (-101.41440 17.60000, -101.41435 17...   
          8082848201 0    LINESTRING (-101.41440 17.60000, -101.41417 17...   
308237995 8097498158 0    LINESTRING (-101.42798 17.60415, -101.42802 17...   
          8085782154 0    LINESTRING (-101.42798 17.60415, -101.42728 17...   

                         bridge      ref junction tunnel access width service   
u         v          key                                                        
308237994 8082848179 0     None  MEX 200     None   None   None  None    None  \
          8085344555 0     None     None     None   None   None  None    None   
          8082848201 0     None  MEX 200     None   None   None  None    None   
308237995 8097498158 0     None  MEX 200     None   None   None  None    None   
          8085782154 0     None  MEX 200     None   None   None  None    None   

                                 city  
u         v          key               
308237994 8082848179 0    Zihuatanejo  
          8085344555 0    Zihuatanejo  
          8082848201 0    Zihuatanejo  
308237995 8097498158 0    Zihuatanejo  
          8085782154 0    Zihuatanejo

In [34]:
# Show
nodes.head(1)

,x,y,street_count,geometry,city
osmid,,,,,
295506951,-101.599217,17.658804,3,POINT (-101.59922 17.65880),Zihuatanejo


In [35]:
nodes_elev_mde, edges_elev_mde = ox.graph_to_gdfs(G, nodes=True, edges=True)

In [36]:
edges_elev_mde

osmid  oneway lanes   
u          v          key                                         
308237994  8082848179 0                 867447567   False     2  \
           8085344555 0                 867141997   False   NaN   
           8082848201 0    [190128099, 867447567]   False     2   
8082848179 8085024461 0                 867447567   False     2   
           308237994  0                 867447567   False     2   
...                                           ...     ...   ...   
9015205779 9015206253 0                 974115861   False   NaN   
           9015206143 0                 974115861   False   NaN   
9015206253 9015205779 0                 974115861   False   NaN   
9015206158 9015206195 0                 974115394   False   NaN   
9015206196 9015206195 0                 974115395   False   NaN   

                                                               name   
u          v          key                                             
308237994  8082848179 0    Carretera Federal Acapulco - Zihuatanejo  \
           8085344555 0                                         NaN   
           8082848201 0    Carretera Federal Acapulco - Zihuatanejo   
8082848179 8085024461 0    Carretera Federal Acapulco - Zihuatanejo   
           308237994  0    Carretera Federal Acapulco - Zihuatanejo   
...                                                             ...   
9015205779 9015206253 0                                         NaN   
           9015206143 0                                         NaN   
9015206253 9015205779 0                                         NaN   
9015206158 9015206195 0                                         NaN   
9015206196 9015206195 0                                         NaN   

                                highway    length   
u          v          key                           
308237994  8082848179 0           trunk   141.252  \
           8085344555 0     residential    15.925   
           8082848201 0           trunk   737.709   
8082848179 8085024461 0           trunk     4.537   
           308237994  0           trunk   141.252   
...                                 ...       ...   
9015205779 9015206253 0    unclassified   209.949   
           9015206143 0    unclassified   585.656   
9015206253 9015205779 0    unclassified   209.949   
9015206158 9015206195 0    unclassified  1405.451   
9015206196 9015206195 0    unclassified   291.932   

                                                                    geometry   
u          v          key                                                      
308237994  8082848179 0    LINESTRING (-101.41440 17.60000, -101.41567 17...  \
           8085344555 0    LINESTRING (-101.41440 17.60000, -101.41435 17...   
           8082848201 0    LINESTRING (-101.41440 17.60000, -101.41417 17...   
8082848179 8085024461 0    LINESTRING (-101.41567 17.60037, -101.41571 17...   
           308237994  0    LINESTRING (-101.41567 17.60037, -101.41440 17...   
...                                                                      ...   
9015205779 9015206253 0    LINESTRING (-101.48461 17.94237, -101.48491 17...   
           9015206143 0    LINESTRING (-101.48461 17.94237, -101.48454 17...   
9015206253 9015205779 0    LINESTRING (-101.48625 17.94286, -101.48611 17...   
9015206158 9015206195 0    LINESTRING (-101.47483 17.97910, -101.47508 17...   
9015206196 9015206195 0    LINESTRING (-101.47263 17.97005, -101.47282 17...   

                               ref         city maxspeed bridge access   
u          v          key                                                
308237994  8082848179 0    MEX 200  Zihuatanejo      NaN    NaN    NaN  \
           8085344555 0        NaN  Zihuatanejo      NaN    NaN    NaN   
           8082848201 0    MEX 200  Zihuatanejo      NaN    NaN    NaN   
8082848179 8085024461 0    MEX 200  Zihuatanejo      NaN    NaN    NaN   
           308237994  0    MEX 200  Zihuatanejo      NaN    NaN    NaN   